In [ ]:
import numpy as np
import pyvista as pv
from kale.algorithms import contour_banded
from kale import theme
import xarray as xr

%config InlineBackend.figure_format = "retina"

# Load Data

In [ ]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.025
TIME_STEP_INTERESTING = 1000

In [ ]:
ds = xr.open_dataset(MESH_VALUES_FILE_NAME, engine="netcdf4")
data = ds["loading_rate"][0, :]
mesh = pv.read(MESH_GEOMETRY_FILE_NAME)
mesh.cell_data["data"] = data
mesh.points[:, -1] *= MESH_Z_SCALE

boundary = mesh.extract_feature_edges(
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)
boundary.clear_data()

# Banded contour for a single time

In [ ]:
pl = pv.Plotter()
pl.add_mesh(
    boundary, color=theme.CONTOUR_LINE_COLOR, line_width=theme.CONTOUR_LINE_WIDTH
)
box = pv.Box(mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
pl.camera.zoom(1.2)

CONTOUR_LEVELS = np.linspace(-60, 60, 21)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="data",
)

pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="data",
    n_colors=N_COLORS,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_OPTS),
)
pl.add_mesh(edges, color=theme.CONTOUR_LINE_COLOR, line_width=theme.CONTOUR_LINE_WIDTH)
pl.show()